In [20]:
!pip install mwclient
!pip install transformers

In [21]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [22]:
import mwclient
import time
from transformers import pipeline
from statistics import mean
import pandas as pd
from datetime import datetime

In [ ]:
site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]
revs = list(page.revisions())
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [ ]:


# Load the Hugging Face sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Looking in indexes: https://download.pytorch.org/whl/cpu


NameError: name 'torch' is not defined

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
import torch
from transformers import pipeline

import torch
from transformers import pipeline

print(torch.__version__)   # to confirm torch is installed

sentiment_pipeline = pipeline("sentiment-analysis")
print(sentiment_pipeline("I love Hugging Face!"))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Looking in indexes: https://download.pytorch.org/whl/cpu
2.8.0+cpu


NameError: name 'torch' is not defined

In [ ]:
import sys
!{sys.executable} -m pip install torch --index-url https://download.pytorch.org/whl/cpu
import torch
from transformers import pipeline

print("Torch version:", torch.__version__)

sentiment_pipeline = pipeline("sentiment-analysis")
print(sentiment_pipeline("I love Hugging Face!"))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Looking in indexes: https://download.pytorch.org/whl/cpu
Torch version: 2.8.0+cpu


NameError: name 'torch' is not defined

In [ ]:
import torch

sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
  sent = sentiment_pipeline([text[:250]])[0]
  score=sent["score"]
  if sent["label"] == "NEGATIVE":
    score *= -1
  return score

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


NameError: name 'torch' is not defined

In [ ]:
edits = {}

for rev in revs:
  date = time.strftime("%Y-%m-%d", rev["timestamp"])

  if date not in edits:
    edits[date] = dict(sentiments=list(), edit_count=0)

  try:
    edits[date]["edit_count"] += 1
    comment = rev["comment"]
    edits[date]["sentiments"].append(find_sentiment(comment))
  except Exception as e:
    continue

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for key in edits:
  if len(edits[key]["sentiments"]) > 0:
    edits[key]["sentiment"] = mean(edits[key]["sentiments"])
    edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
  else:
    edits[key]["sentiment"] = 0
    edits[key]["neg_sentiment"] = 0

  del edits[key]["sentiments"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
edits_df = pd.DataFrame.from_dict(edits, orient="index")
edits_df.index = pd.to_datetime(edits_df.index)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dates = pd.date_range(start="2009-03-08", end=datetime.today())

edits_df =edits_df.reindex(dates, fill_value=0)
rolling_edits = edits_df.rolling(30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits.to_csv("wikipedia_edits.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
